In [27]:
import pandas as pd
import numpy as np
import scipy.stats as sci

data = pd.read_excel("proshares_analysis_data.xlsx", sheet_name="hedge_fund_series",index_col=0)
# Forward-fill missing return value at end of HFRIFWI series
data["HFRIFWI Index"] = data["HFRIFWI Index"].ffill()

mean = data.mean() * 12
vol = np.sqrt(data.var() * 12)
sharpe = mean / vol
print("Annualized Mean, Volatility, and Sharpe of Returns")
display(mean, vol, sharpe)

Annualized Mean, Volatility, and Sharpe of Returns


HFRIFWI Index     0.052538
MLEIFCTR Index    0.038535
MLEIFCTX Index    0.036526
HDG US Equity     0.026880
QAI US Equity     0.028811
dtype: float64

HFRIFWI Index     0.058813
MLEIFCTR Index    0.055240
MLEIFCTX Index    0.055102
HDG US Equity     0.057388
QAI US Equity     0.049823
dtype: float64

HFRIFWI Index     0.893298
MLEIFCTR Index    0.697589
MLEIFCTX Index    0.662880
HDG US Equity     0.468396
QAI US Equity     0.578275
dtype: float64

In [38]:
def drawdown_from_returns(df):
    results = {}

    for col in df.columns:
        s = df[col].dropna()
        if s.empty:
            results[col] = [0.0, None, None, None]
            continue

        cumulative = (1 + s).cumprod()
        running_max = cumulative.cummax()
        drawdown = cumulative / running_max - 1.0

        trough = drawdown.idxmin()
        peak_level = running_max.loc[trough]
        peak = running_max.loc[:trough]
        peak = peak[peak == peak_level].index[-1]

        post = cumulative.loc[trough:]
        rec_hits = post[post >= peak_level]
        recovery = rec_hits.index[0] if len(rec_hits) > 0 else None

        results[col] = [float(drawdown.min()), peak, trough, recovery]

    return pd.DataFrame(results, index=["Max Drawdown", "Peak Date", "Trough Date", "Recovery Date"]).T

In [40]:
skew = sci.skew(data)
kurtosis = sci.kurtosis(data, fisher=True)
VaR = data.quantile(0.05)
CVaR = data[data <= VaR].mean()
drawdowns = drawdown_from_returns(data)
skew = pd.Series(skew, index=mean.index)
kurtosis = pd.Series(kurtosis, index=mean.index)
VaR = pd.Series(VaR, index=mean.index)
CVaR = pd.Series(CVaR, index=mean.index)
print("Skew, Excess Kurtosis, VaR, CVaR, and Max Drawdowns")
display(skew, kurtosis, VaR, CVaR, drawdowns)

Skew, Excess Kurtosis, VaR, CVaR, and Max Drawdowns


HFRIFWI Index    -0.945171
MLEIFCTR Index   -0.287417
MLEIFCTX Index   -0.271058
HDG US Equity    -0.272473
QAI US Equity    -0.429674
dtype: float64

HFRIFWI Index     5.426550
MLEIFCTR Index    1.548255
MLEIFCTX Index    1.508328
HDG US Equity     1.689572
QAI US Equity     1.371823
dtype: float64

HFRIFWI Index    -0.023997
MLEIFCTR Index   -0.027002
MLEIFCTX Index   -0.026970
HDG US Equity    -0.029919
QAI US Equity    -0.017171
Name: 0.05, dtype: float64

HFRIFWI Index    -0.035992
MLEIFCTR Index   -0.034993
MLEIFCTX Index   -0.034937
HDG US Equity    -0.036831
QAI US Equity    -0.030995
dtype: float64

,Max Drawdown,Peak Date,Trough Date,Recovery Date
HFRIFWI Index,-0.115473,2020-03-31 00:00:00,2020-03-31 00:00:00,2020-08-31 00:00:00
MLEIFCTR Index,-0.124302,2022-09-30 00:00:00,2022-09-30 00:00:00,2024-02-29 00:00:00
MLEIFCTX Index,-0.124388,2022-09-30 00:00:00,2022-09-30 00:00:00,2024-02-29 00:00:00
HDG US Equity,-0.14072,2022-09-30 00:00:00,2022-09-30 00:00:00,2024-07-31 00:00:00
QAI US Equity,-0.137716,2022-09-30 00:00:00,2022-09-30 00:00:00,2024-02-29 00:00:00
